In [1]:
import pandas as pd
import numpy as np
import sqlite3
import urllib
import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
# 드라이버 객체 생성
driver = webdriver.Chrome('chromedriver')

In [3]:
# DB 생성
conn = sqlite3.connect('meatbox.db')
cur = conn.cursor()

In [ ]:
#cur.execute('drop table 상품코드정보')
#cur.execute('drop table 상세정보')
#cur.execute('drop table 판매자정보')
#cur.execute('drop table 상품정보')
#cur.execute('drop table 판매정보')

In [131]:
# 테이블 내용물 삭제
#cur.execute('delete from 상품정보')
#cur.execute('delete from 상세정보')
#cur.execute('delete from 판매자정보')

In [4]:
# 테이블 생성
cur.execute('create table 상품코드정보(기준일자, 상품코드 text, primary key(기준일자, 상품코드))')
cur.execute('create table 상세정보(상품코드 text, 보관방법, 축종, 품목, 가공방법, 원산지, 브랜드, 등급, 배송불가지역 ,배송정책, 판매대상, EST_NO, 포장상태, primary key(상품코드))')
cur.execute('create table 판매자정보(상품코드 text, 대표자, 상호, 전화, 이메일주소, 사업자등록번호, 사업장소재지, primary key(상품코드))')
cur.execute('create table 상품정보(상품코드 text, kg당가격, 가공일, 유통기한, 최대, 최소, 평균, 유통식별번호, primary key(상품코드))')
cur.execute('create table 판매정보(기준일자, 상품코드 text, 구매, 후기, primary key(기준일자, 상품코드))')

In [5]:
#### BeautifulSoup를 이용해 리팩토링 요망 ####
# 상품코드 수집
def 상품코드수집():
    date = datetime.date.today()
    url = 'https://www.meatbox.co.kr/fo/product/productListPage.do'
    params = {'categorySeq':10005,'articleCount':'1000'}
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    driver.get(url_params)
    상품코드 = driver.find_elements_by_xpath("//li[@data-product-seq]")
    상품코드정보 = []
    for 상품 in 상품코드:
        상품코드정보.append((str(datetime.date.today()), 상품.get_attribute('data-product-seq')))
    return 상품코드정보

In [13]:
def 정보수집(코드):
    기준일자 = str(datetime.date.today())
    # 타겟 접근
    url = 'https://www.meatbox.co.kr/fo/product/productViewPage.do'
    params = {'productSeq':코드}
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    driver.get(url_params)
    
    # 상세정보수집
    테이블 = driver.find_element_by_xpath("//table[@class='section_td']").find_elements_by_tag_name('tr')
    상세정보 = {'상품코드':코드}
    for 레코드 in 테이블:
        속성 = 레코드.find_element_by_tag_name('th').text
        내용 = 레코드.find_element_by_tag_name('td').text
        상세정보[속성] = 내용
        
    # 판매정보수집
    테이블 = driver.find_element_by_xpath("//li[@class='bott_info']").find_elements_by_tag_name('dl')
    판매정보 = {'기준일자':기준일자, '상품코드':코드}
    for 레코드 in 테이블:
        속성 = 레코드.find_element_by_tag_name('dt').get_attribute('innerHTML')
        내용 = 레코드.find_element_by_tag_name('em').get_attribute('innerHTML')
        판매정보[속성] = 내용
    
    # 판매자정보수집
    테이블 = driver.find_element_by_xpath("//div[@class='pop_comm sale_pop']").find_elements_by_tag_name('tr')
    판매자정보 = {'상품코드':코드}
    for 레코드 in 테이블:
        속성 = 레코드.find_element_by_tag_name('th').get_attribute('innerHTML')
        내용 = 레코드.find_element_by_tag_name('td').get_attribute('innerHTML')
        판매자정보[속성] = 내용
    판매자정보['사업자등록번호'] = 판매자정보['사업자등록번호'].split('<strong>')[0] 
    
    # 상품정보수집
    try:
        테이블 = driver.find_element_by_xpath("//div[@class='pop_comm deteil_over_pop _optionDetailInfoLayer']").find_element_by_xpath("//div[@class='table_wrap']").find_elements_by_tag_name('tr')
        상품정보 = {'상품코드':코드}
        for 레코드 in 테이블:
            속성 = 레코드.find_element_by_tag_name('th').get_attribute('innerHTML')
            내용 = 레코드.find_element_by_tag_name('td').get_attribute('innerHTML')
            상품정보[속성] = 내용
    except NoSuchElementException:
        상품정보 = {'상품코드':코드}
        
    return (상세정보, 판매정보, 판매자정보, 상품정보)

In [14]:
def 데이터적재(상세정보, 판매정보, 판매자정보, 상품정보):
    # 데이터 가공
    df = pd.DataFrame(columns=['상품코드','보관방법','축종','품목','가공방법','원산지','브랜드','등급','배송불가지역','배송정책','판매대상','EST NO','포장상태'])
    df = df.append(상세정보, ignore_index=True)
    상세정보 = tuple(df.loc[0])
    
    df = pd.DataFrame(columns=['기준일자','상품코드','구매','후기'])
    df = df.append(판매정보, ignore_index=True)
    판매정보 = tuple(df.loc[0])
    
    df = pd.DataFrame(columns=['상품코드','대표자','상호','전화','이메일 주소','사업자등록번호','사업장 소재지'])
    df = df.append(판매자정보, ignore_index=True)
    판매자정보 = tuple(df.loc[0])
    
    df = pd.DataFrame(columns=['상품코드','kg당 가격','가공일','유통기한','최대','최소','평균','유통식별번호'])
    df = df.append(상품정보, ignore_index=True)
    상품정보 = tuple(df.loc[0])
    
    # 데이터 적재
    try:
        cur.execute('insert into 상세정보 values (?,?,?,?,?,?,?,?,?,?,?,?,?)', 상세정보) 
        cur.execute('insert into 판매정보 values (?,?,?,?)', 판매정보)
        cur.execute('insert into 판매자정보 values (?,?,?,?,?,?,?)', 판매자정보)
        cur.execute('insert into 상품정보 values (?,?,?,?,?,?,?,?)', 상품정보)
        print('적재성공(상품코드 : {:})'.format(상세정보[0]))
    except sqlite3.IntegrityError:
        print('적재실패(상품코드 : {:}) : 무결성 조건 오류'.format(상세정보[0]))
    return None

In [15]:
def 적재대상코드(기준일자):
    sql = "select 상품코드 from 상품코드정보 where 기준일자 = '{}' and 상품코드 not in (select 상품코드 from 판매정보 where 기준일자 = '{}')".format(기준일자, 기준일자)
    cur.execute(sql)
    rows = list(map(lambda x:x[0], cur.fetchall()))
    return rows

In [ ]:
# 상품코드 수집 
상품코드정보 = 상품코드수집() 
sql = 'insert into 상품코드정보 values (?,?)'
cur.executemany(sql, 상품코드정보)

In [20]:
# 적재대상코드 추출
기준일자 = str(datetime.date.today())
상품코드 = 적재대상코드(기준일자)

In [ ]:
# 상품정보 수집 및 데이터 적재
for 코드 in 상품코드:
    상세정보, 판매정보, 판매자정보, 상품정보 = 정보수집(코드)
    데이터적재(상세정보, 판매정보, 판매자정보, 상품정보)

적재성공(상품코드 : 112092)
적재성공(상품코드 : 112117)
적재성공(상품코드 : 112142)
적재성공(상품코드 : 112158)
